# Graph attendees

Download the Readable Data spreadsheet as a CSV and save it as `attendees.csv` in the same directory as this notebook.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl
import community as com
import csv

attendees = {}

with open('attendees.csv') as attendees_csv:
    csv_reader = csv.reader(attendees_csv, delimiter=',')
    next(csv_reader, None)
    for row in csv_reader:
        attendees[row[6]] = [] + ([] if not row[11] else [row[11]]) + ([] if not row[12] else [row[12]])

Create the graph for the attendees using `networkx`. Each attendee is a node and the edges are the preferences. The weight of each edge indicates whether it is a one-way preference, or two-way preference.

In [ ]:
G = nx.DiGraph()
G.add_nodes_from(attendees.keys())

for attendee, preferences in attendees.items():
    for preference in preferences:
        if preference in attendees and attendee in attendees[preference]:
            G.add_edge(attendee, preference, weight=2)
        else:
            G.add_edge(attendee, preference, weight=1)

Print all subgraphs of the graph.

In [ ]:
for index, SG in enumerate(nx.connected_components(G.to_undirected())):
    print(SG)

Plot the graph using `matplotlib`.

In [ ]:
degrees = dict(G.degree)
nodes = G.nodes()
edge_labels = nx.get_edge_attributes(G,'weight')
pos = nx.nx_pydot.graphviz_layout(G)

Use Louvain community detection to find the communities in the graph and colour the nodes accordingly.

In [ ]:
partition = com.best_partition(G.to_undirected(), resolution=0.2)
cmap = mpl.colormaps['hsv']

Plot everything, colouring the labels based on whether the attendee has a ticket, and colouring the nodes based on its community.

In [ ]:
plt.figure(figsize=(30, 30))
plt.axis('off')

nx.draw_networkx_edges(G, pos, width=1) 
nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=7)

nx.draw_networkx_nodes(G, pos, partition.keys(), 
                       cmap=cmap, node_color=list(partition.values()),
                       node_size=[100 + v * 100 for v in degrees.values()],
                       alpha=0.8)

for node, (x, y) in pos.items():
    if node in attendees:
        plt.text(x, y - 0.1, node, ha='center', va='center', color='#000000', fontsize=10)
    else:
        plt.text(x, y - 0.1, node, ha='center', va='center', color='#ff0000', fontsize=10)

plt.savefig('graph.png', dpi=300, bbox_inches='tight')
plt.show()